<a href="https://colab.research.google.com/github/Janina712/MLTSA22_JBoecher/blob/main/Other/BreathGroup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**0. Imports & Set-Up**

In [82]:
import pandas as pd
import numpy as np 
import os

In [83]:
from google.colab import drive
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [186]:
%cd /content/gdrive/MyDrive/RhythmAnalysisPipeline/1.CombinedInputFiles/

/content/gdrive/MyDrive/RhythmAnalysisPipeline/1.CombinedInputFiles


In [187]:
read_TextGrid_duration = pd.read_excel("read_TextGrid_comb.xlsx")
df_read_txt_wide = pd.read_excel("read_txt_comb.xlsx")

**1. Create empty dataframe row to store BG assignment**

In [188]:
BG_col = pd.DataFrame()

**2. Create variables needed inside loop**

In [189]:
IDs = ['24fa', '24ma', '24mb', '26f', '29ma', '29mb', '29mc', '32m', '34m', '35mb', '46ma', '50fa', '50fb', '54f', '57m', '60m', '62f', '62m', 'C10_KS', 'C11_NP', 'C12_CC', 'C13_RG','C1_DS', 'C2_JH', 'C3_MD', 'C4_ZO', 'C5_HL', 'C6_LH', 'C7_SH', 'C9_RD']

In [190]:
IntervalTimes = df_read_txt_wide[['ID', 'Offset']] 

In [191]:
Offsets_all = read_TextGrid_duration[['ID', 'Offset']]

In [192]:
ID_freq_sound = read_TextGrid_duration['ID'].value_counts().sort_index()

**3. Assign Breath Groups for All Participants in a Loop**

In [193]:
n = -1

In [194]:
for ID in IDs:
  n = n + 1
  BG_current = pd.DataFrame(index=range(int(ID_freq_sound[n])),columns=["Breath.Group"])
  subset_IT = IntervalTimes[IntervalTimes["ID"] == ID]
  subset_IT.index = range(len(subset_IT.index))
  subset_sounds = Offsets_all[Offsets_all["ID"] == ID]
  subset_sounds.index = range(len(subset_sounds.index))
  for j in range(0,len(subset_IT)): # index over range of utterances from participant
    for i in range(0,len(subset_sounds)-1): # index over sounds from participant
      if subset_sounds["Offset"][i] <= subset_IT["Offset"][j]:
        if pd.isna(BG_current["Breath.Group"][i]) == True:
          BG_current["Breath.Group"][i] = j+1 # assign number of breath group to corresponding row
  BG_col = BG_col.append([BG_current], ignore_index=True)


In [195]:
read_TextGrid_duration = pd.concat([read_TextGrid_duration, BG_col], axis=1)

In [197]:
read_TextGrid_duration.drop(['Unnamed: 0'], axis=1 , inplace = True)

In [198]:
read_TextGrid_duration

,ID,Type,Sound,Onset,Offset,Duration,Breath.Group
0,24fa,silence,"""sp""",0.000,0.013,0.013,1
1,24fa,silence,"""sp""",0.013,0.143,0.130,1
2,24fa,consonant,"""S""",0.143,0.263,0.120,1
3,24fa,vowel,"""OW1""",0.263,0.343,0.080,1
4,24fa,consonant,"""HH""",0.343,0.482,0.139,1
...,...,...,...,...,...,...,...
24338,C13_RG,consonant,"""N""",87.881,87.910,0.029,28
24339,C13_RG,vowel,"""EY2""",87.910,88.081,0.171,28
24340,C13_RG,consonant,"""M""",88.081,88.191,0.110,28
24341,C13_RG,consonant,"""Z""",88.191,88.281,0.090,28


**4. Count Number of Sounds in a BreathGroup (m)**

In [199]:
m_col = pd.DataFrame()  ## initialize group-level dataframe
for ID in IDs: ## loop over participnts
  m_current_ID = pd.DataFrame()   ## initialize participant-level dataframe
  subset_sounds = read_TextGrid_duration[read_TextGrid_duration["ID"] == ID]  # get subset of sound dataframe that corresponds to current participant
  subset_sounds.index = range(len(subset_sounds.index)) # reset index
  m = subset_sounds["Breath.Group"].value_counts().sort_index() # count how often a certain Breath group occurs for this participant
  m.index = range(len(m.index)) # reset index
  for a in range (0,len(m)): # go through all breath groups that this participant produced
    m_current_BG = pd.DataFrame()  ## initialize BG-level dataframe
    m_current_BG = pd.DataFrame(np.repeat(m.iloc[a], m.iloc[a], axis=0)) #replicate the sum sum times
    m_current_ID = m_current_ID.append([m_current_BG], ignore_index = True) # add BG-level dataframe to participant-level dataframe 
  m_col = m_col.append([m_current_ID], ignore_index = True) # add participant-level dataframe to group-level dataframe

In [202]:
read_TextGrid_duration = read_TextGrid_duration.dropna() # drop NaNs to make it match with m_col

In [204]:
read_TextGrid_duration.index = range(len(read_TextGrid_duration.index)) # reset index

In [206]:
read_TextGrid_duration = pd.concat([read_TextGrid_duration, m_col], axis=1) # concatenate main df with m_col

In [207]:
read_TextGrid_duration.rename(columns = {0:'m'}, inplace = True) # rename new column

In [208]:
read_TextGrid_duration

,ID,Type,Sound,Onset,Offset,Duration,Breath.Group,m
0,24fa,silence,"""sp""",0.000,0.013,0.013,1,33
1,24fa,silence,"""sp""",0.013,0.143,0.130,1,33
2,24fa,consonant,"""S""",0.143,0.263,0.120,1,33
3,24fa,vowel,"""OW1""",0.263,0.343,0.080,1,33
4,24fa,consonant,"""HH""",0.343,0.482,0.139,1,33
...,...,...,...,...,...,...,...,...
24281,C13_RG,vowel,"""ER1""",87.691,87.881,0.190,28,38
24282,C13_RG,consonant,"""N""",87.881,87.910,0.029,28,38
24283,C13_RG,vowel,"""EY2""",87.910,88.081,0.171,28,38
24284,C13_RG,consonant,"""M""",88.081,88.191,0.110,28,38


**5. Save Results Created by Loop**

In [183]:
%cd /content/gdrive/MyDrive/RhythmAnalysisPipeline/

/content/gdrive/MyDrive/RhythmAnalysisPipeline


In [103]:
dir = "2.BreathGroups_Assigned"

if os.path.exists(dir) == False:
  os.mkdir(dir)

In [209]:
%cd /content/gdrive/MyDrive/RhythmAnalysisPipeline/2.BreathGroups_Assigned/

/content/gdrive/MyDrive/RhythmAnalysisPipeline/2.BreathGroups_Assigned


In [210]:
read_TextGrid_duration.to_excel("reading_TextGrid_comb_BG_loop.xlsx")

In [106]:
%cd /content/gdrive/MyDrive/RhythmAnalysisPipeline/

/content/gdrive/MyDrive/RhythmAnalysisPipeline


# **Go Through Participants Individually**

**1. Assign Breath Groups for one Participant at a Time**

In [28]:
BG_24fa = pd.DataFrame(index=range(int(ID_freq_sound[0])),columns=["Breath.Group"])
subset_IT = IntervalTimes[IntervalTimes["ID"] == "24fa"]
subset_IT.index = range(len(subset_IT.index))
subset_sounds = Offsets_all[Offsets_all["ID"] == "24fa"]
subset_sounds.index = range(len(subset_sounds.index))
for j in range(0,len(subset_IT)):
  for i in range(0,len(subset_sounds)-1):
    if subset_sounds["Offset"][i] <= subset_IT["Offset"][j]:
      if pd.isna(BG_24fa["Breath.Group"][i]) == True:
        BG_24fa["Breath.Group"][i] = j+1

In [203]:
m_24fa = pd.DataFrame()  
m = BG_24fa["Breath.Group"].value_counts().sort_index()
subset_IT = IntervalTimes[IntervalTimes["ID"] == "24fa"]
subset_IT.index = range(len(subset_IT.index))
for a in range (0,len(subset_IT)):
  m_current = pd.DataFrame(np.repeat(m.iloc[a], m.iloc[a], axis=0))
  m_24fa = m_24fa.append([m_current], ignore_index = True)
BG_24fa = pd.concat([BG_24fa, m_24fa], axis=1).astype('Int64')
BG_24fa.rename(columns = {0:'m'}, inplace = True)

In [208]:
BG_24ma = pd.DataFrame(index=range(int(ID_freq_sound[1])),columns = ["Breath.Group"])
subset_IT = IntervalTimes[IntervalTimes["ID"] == "24ma"]
subset_IT.index = range(len(subset_IT.index))
subset_sounds = Offsets_all[Offsets_all["ID"] == "24ma"]
subset_sounds.index = range(len(subset_sounds.index))
for j in range(0,len(subset_IT)):
  for i in range(0,len(subset_sounds)-1):
    if subset_sounds["Offset"][i] <= subset_IT["Offset"][j]:
      if pd.isna(BG_24ma["Breath.Group"][i]) == True:
        BG_24ma["Breath.Group"][i] = j+1

In [209]:
BG_24mb = pd.DataFrame(index=range(int(ID_freq_sound[2])),columns=["Breath.Group"])
subset_IT = IntervalTimes[IntervalTimes["ID"] == "24mb"]
subset_IT.index = range(len(subset_IT.index))
subset_sounds = Offsets_all[Offsets_all["ID"] == "24mb"]
subset_sounds.index = range(len(subset_sounds.index))
for j in range(0,len(subset_IT)):
  for i in range(0,len(subset_sounds)-1):
    if subset_sounds["Offset"][i] <= subset_IT["Offset"][j]:
      if pd.isna(BG_24mb["Breath.Group"][i]) == True:
        BG_24mb["Breath.Group"][i] = j+1

In [210]:
BG_26f = pd.DataFrame(index=range(int(ID_freq_sound[3])),columns=["Breath.Group"])
subset_IT = IntervalTimes[IntervalTimes["ID"] == "26f"]
subset_IT.index = range(len(subset_IT.index))
subset_sounds = Offsets_all[Offsets_all["ID"] == "26f"]
subset_sounds.index = range(len(subset_sounds.index))
for j in range(0,len(subset_IT)):
  for i in range(0,len(subset_sounds)-1):
    if subset_sounds["Offset"][i] <= subset_IT["Offset"][j]:
      if pd.isna(BG_26f["Breath.Group"][i]) == True:
        BG_26f["Breath.Group"][i] = j+1

In [211]:
BG_29ma = pd.DataFrame(index=range(int(ID_freq_sound[4])),columns=["Breath.Group"])     # range(1)
subset_IT = IntervalTimes[IntervalTimes["ID"] == "29ma"]
subset_IT.index = range(len(subset_IT.index))
subset_sounds = Offsets_all[Offsets_all["ID"] == "29ma"]
subset_sounds.index = range(len(subset_sounds.index))
for j in range(0,len(subset_IT)):
  for i in range(0,len(subset_sounds)-1):
    if subset_sounds["Offset"][i] <= subset_IT["Offset"][j]:
      if pd.isna(BG_29ma["Breath.Group"][i]) == True:
        BG_29ma["Breath.Group"][i] = j+1

In [212]:
BG_29mb = pd.DataFrame(index=range(int(ID_freq_sound[5])),columns=["Breath.Group"])
subset_IT = IntervalTimes[IntervalTimes["ID"] == "29mb"]
subset_IT.index = range(len(subset_IT.index))
subset_sounds = Offsets_all[Offsets_all["ID"] == "29mb"]
subset_sounds.index = range(len(subset_sounds.index))
for j in range(0,len(subset_IT)):
  for i in range(0,len(subset_sounds)-1):
    if subset_sounds["Offset"][i] <= subset_IT["Offset"][j]:
      if pd.isna(BG_29mb["Breath.Group"][i]) == True:
        BG_29mb["Breath.Group"][i] = j+1

In [213]:
BG_29mc = pd.DataFrame(index=range(int(ID_freq_sound[6])),columns=["Breath.Group"])
subset_IT = IntervalTimes[IntervalTimes["ID"] == "29mc"]
subset_IT.index = range(len(subset_IT.index))
subset_sounds = Offsets_all[Offsets_all["ID"] == "29mc"]
subset_sounds.index = range(len(subset_sounds.index))
for j in range(0,len(subset_IT)):
  for i in range(0,len(subset_sounds)-1):
    if subset_sounds["Offset"][i] <= subset_IT["Offset"][j]:
      if pd.isna(BG_29mc["Breath.Group"][i]) == True:
        BG_29mc["Breath.Group"][i] = j+1

In [214]:
BG_32m = pd.DataFrame(index=range(int(ID_freq_sound[7])),columns=["Breath.Group"])
subset_IT = IntervalTimes[IntervalTimes["ID"] == "32m"]
subset_IT.index = range(len(subset_IT.index))
subset_sounds = Offsets_all[Offsets_all["ID"] == "32m"]
subset_sounds.index = range(len(subset_sounds.index))
for j in range(0,len(subset_IT)):
  for i in range(0,len(subset_sounds)-1):
    if subset_sounds["Offset"][i] <= subset_IT["Offset"][j]:
      if pd.isna(BG_32m["Breath.Group"][i]) == True:
        BG_32m["Breath.Group"][i] = j+1

In [215]:
BG_34m = pd.DataFrame(index=range(int(ID_freq_sound[8])),columns=["Breath.Group"])
subset_IT = IntervalTimes[IntervalTimes["ID"] == "34m"]
subset_IT.index = range(len(subset_IT.index))
subset_sounds = Offsets_all[Offsets_all["ID"] == "34m"]
subset_sounds.index = range(len(subset_sounds.index))
for j in range(0,len(subset_IT)):
  for i in range(0,len(subset_sounds)-1):
    if subset_sounds["Offset"][i] <= subset_IT["Offset"][j]:
      if pd.isna(BG_34m["Breath.Group"][i]) == True:
        BG_34m["Breath.Group"][i] = j+1

In [216]:
BG_35mb = pd.DataFrame(index=range(int(ID_freq_sound[9])),columns=["Breath.Group"])
subset_IT = IntervalTimes[IntervalTimes["ID"] == "35mb"]
subset_IT.index = range(len(subset_IT.index))
subset_sounds = Offsets_all[Offsets_all["ID"] == "35mb"]
subset_sounds.index = range(len(subset_sounds.index))
for j in range(0,len(subset_IT)):
  for i in range(0,len(subset_sounds)-1):
    if subset_sounds["Offset"][i] <= subset_IT["Offset"][j]:
      if pd.isna(BG_35mb["Breath.Group"][i]) == True:
        BG_35mb["Breath.Group"][i] = j+1

In [217]:
BG_46ma = pd.DataFrame(index=range(int(ID_freq_sound[10])),columns=["Breath.Group"])
subset_IT = IntervalTimes[IntervalTimes["ID"] == "46ma"]
subset_IT.index = range(len(subset_IT.index))
subset_sounds = Offsets_all[Offsets_all["ID"] == "46ma"]
subset_sounds.index = range(len(subset_sounds.index))
for j in range(0,len(subset_IT)):
  for i in range(0,len(subset_sounds)-1):
    if subset_sounds["Offset"][i] <= subset_IT["Offset"][j]:
      if pd.isna(BG_46ma["Breath.Group"][i]) == True:
        BG_46ma["Breath.Group"][i] = j+1

In [218]:
BG_50fa = pd.DataFrame(index=range(int(ID_freq_sound[11])),columns=["Breath.Group"])
subset_IT = IntervalTimes[IntervalTimes["ID"] == "50fa"]
subset_IT.index = range(len(subset_IT.index))
subset_sounds = Offsets_all[Offsets_all["ID"] == "50fa"]
subset_sounds.index = range(len(subset_sounds.index))
for j in range(0,len(subset_IT)):
  for i in range(0,len(subset_sounds)-1):
    if subset_sounds["Offset"][i] <= subset_IT["Offset"][j]:
      if pd.isna(BG_50fa["Breath.Group"][i]) == True:
        BG_50fa["Breath.Group"][i] = j+1

In [219]:
BG_50fb = pd.DataFrame(index=range(int(ID_freq_sound[12])),columns=["Breath.Group"])
subset_IT = IntervalTimes[IntervalTimes["ID"] == "50fb"]
subset_IT.index = range(len(subset_IT.index))
subset_sounds = Offsets_all[Offsets_all["ID"] == "50fb"]
subset_sounds.index = range(len(subset_sounds.index))
for j in range(0,len(subset_IT)):
  for i in range(0,len(subset_sounds)-1):
    if subset_sounds["Offset"][i] <= subset_IT["Offset"][j]:
      if pd.isna(BG_50fb["Breath.Group"][i]) == True:
        BG_50fb["Breath.Group"][i] = j+1

In [220]:
BG_54f = pd.DataFrame(index=range(int(ID_freq_sound[13])),columns=["Breath.Group"])
subset_IT = IntervalTimes[IntervalTimes["ID"] == "54f"]
subset_IT.index = range(len(subset_IT.index))
subset_sounds = Offsets_all[Offsets_all["ID"] == "54f"]
subset_sounds.index = range(len(subset_sounds.index))
for j in range(0,len(subset_IT)):
  for i in range(0,len(subset_sounds)-1):
    if subset_sounds["Offset"][i] <= subset_IT["Offset"][j]:
      if pd.isna(BG_54f["Breath.Group"][i]) == True:
        BG_54f["Breath.Group"][i] = j+1

In [221]:
BG_57m = pd.DataFrame(index=range(int(ID_freq_sound[14])),columns=["Breath.Group"])
subset_IT = IntervalTimes[IntervalTimes["ID"] == "57m"]
subset_IT.index = range(len(subset_IT.index))
subset_sounds = Offsets_all[Offsets_all["ID"] == "57m"]
subset_sounds.index = range(len(subset_sounds.index))
for j in range(0,len(subset_IT)):
  for i in range(0,len(subset_sounds)-1):
    if subset_sounds["Offset"][i] <= subset_IT["Offset"][j]:
      if pd.isna(BG_57m["Breath.Group"][i]) == True:
        BG_57m["Breath.Group"][i] = j+1

In [222]:
BG_60m = pd.DataFrame(index=range(int(ID_freq_sound[15])),columns= ["Breath.Group"])
subset_IT = IntervalTimes[IntervalTimes["ID"] == "60m"]
subset_IT.index = range(len(subset_IT.index))
subset_sounds = Offsets_all[Offsets_all["ID"] == "60m"]
subset_sounds.index = range(len(subset_sounds.index))
for j in range(0,len(subset_IT)):
  for i in range(0,len(subset_sounds)-1):
    if subset_sounds["Offset"][i] <= subset_IT["Offset"][j]:
      if pd.isna(BG_60m["Breath.Group"][i]) == True:
        BG_60m["Breath.Group"][i] = j+1

In [223]:
BG_62f = pd.DataFrame(index=range(int(ID_freq_sound[16])),columns=["Breath.Group"])
subset_IT = IntervalTimes[IntervalTimes["ID"] == "62f"]
subset_IT.index = range(len(subset_IT.index))
subset_sounds = Offsets_all[Offsets_all["ID"] == "62f"]
subset_sounds.index = range(len(subset_sounds.index))
for j in range(0,len(subset_IT)):
  for i in range(0,len(subset_sounds)-1):
    if subset_sounds["Offset"][i] <= subset_IT["Offset"][j]:
      if pd.isna(BG_62f["Breath.Group"][i]) == True:
        BG_62f["Breath.Group"][i] = j+1

In [224]:
BG_62m = pd.DataFrame(index=range(int(ID_freq_sound[17])),columns=["Breath.Group"])
subset_IT = IntervalTimes[IntervalTimes["ID"] == "62m"]
subset_IT.index = range(len(subset_IT.index))
subset_sounds = Offsets_all[Offsets_all["ID"] == "62m"]
subset_sounds.index = range(len(subset_sounds.index))
for j in range(0,len(subset_IT)):
  for i in range(0,len(subset_sounds)-1):
    if subset_sounds["Offset"][i] <= subset_IT["Offset"][j]:
      if pd.isna(BG_62m["Breath.Group"][i]) == True:
        BG_62m["Breath.Group"][i] = j+1

In [225]:
BG_C10_KS = pd.DataFrame(index=range((int(ID_freq_sound[18]))),columns = ["Breath.Group"])
subset_IT = IntervalTimes[IntervalTimes["ID"] == "C10_KS"]
subset_IT.index = range(len(subset_IT.index))
subset_sounds = Offsets_all[Offsets_all["ID"] == "C10_KS"]
subset_sounds.index = range(len(subset_sounds.index))
for j in range(0,len(subset_IT)):
  for i in range(0,len(subset_sounds)-1):
    if subset_sounds["Offset"][i] <= subset_IT["Offset"][j]:
      if pd.isna(BG_C10_KS["Breath.Group"][i]) == True:
        BG_C10_KS["Breath.Group"][i] = j+1

In [226]:
BG_C11_NP = pd.DataFrame(index=range(int(ID_freq_sound[19])),columns=["Breath.Group"])
subset_IT = IntervalTimes[IntervalTimes["ID"] == "C11_NP"]
subset_IT.index = range(len(subset_IT.index))
subset_sounds = Offsets_all[Offsets_all["ID"] == "C11_NP"]
subset_sounds.index = range(len(subset_sounds.index))
for j in range(0,len(subset_IT)):
  for i in range(0,len(subset_sounds)-1):
    if subset_sounds["Offset"][i] <= subset_IT["Offset"][j]:
      if pd.isna(BG_C11_NP["Breath.Group"][i]) == True:
        BG_C11_NP["Breath.Group"][i] = j+1

In [227]:
BG_C12_CC = pd.DataFrame(index=range(int(ID_freq_sound[20])),columns=["Breath.Group"])
subset_IT = IntervalTimes[IntervalTimes["ID"] == "C12_CC"]
subset_IT.index = range(len(subset_IT.index))
subset_sounds = Offsets_all[Offsets_all["ID"] == "C12_CC"]
subset_sounds.index = range(len(subset_sounds.index))
for j in range(0,len(subset_IT)):
  for i in range(0,len(subset_sounds)-1):
    if subset_sounds["Offset"][i] <= subset_IT["Offset"][j]:
      if pd.isna(BG_C12_CC["Breath.Group"][i]) == True:
        BG_C12_CC["Breath.Group"][i] = j+1

In [228]:
BG_C13_RG = pd.DataFrame(index=range(int(ID_freq_sound[21])),columns=["Breath.Group"])
subset_IT = IntervalTimes[IntervalTimes["ID"] == "C13_RG"]
subset_IT.index = range(len(subset_IT.index))
subset_sounds = Offsets_all[Offsets_all["ID"] == "C13_RG"]
subset_sounds.index = range(len(subset_sounds.index))
for j in range(0,len(subset_IT)):
  for i in range(0,len(subset_sounds)-1):
    if subset_sounds["Offset"][i] <= subset_IT["Offset"][j]:
      if pd.isna(BG_C13_RG["Breath.Group"][i]) == True:
        BG_C13_RG["Breath.Group"][i] = j+1

In [229]:
BG_C1_DS = pd.DataFrame(index=range(int(ID_freq_sound[22])),columns=["Breath.Group"])
subset_IT = IntervalTimes[IntervalTimes["ID"] == "C1_DS"]
subset_IT.index = range(len(subset_IT.index))
subset_sounds = Offsets_all[Offsets_all["ID"] == "C1_DS"]
subset_sounds.index = range(len(subset_sounds.index))
for j in range(0,len(subset_IT)):
  for i in range(0,len(subset_sounds)-1):
    if subset_sounds["Offset"][i] <= subset_IT["Offset"][j]:
      if pd.isna(BG_C1_DS["Breath.Group"][i]) == True:
        BG_C1_DS["Breath.Group"][i] = j+1

In [230]:
BG_C2_JH = pd.DataFrame(index=range(int(ID_freq_sound[23])),columns=["Breath.Group"])
subset_IT = IntervalTimes[IntervalTimes["ID"] == "C2_JH"]
subset_IT.index = range(len(subset_IT.index))
subset_sounds = Offsets_all[Offsets_all["ID"] == "C2_JH"]
subset_sounds.index = range(len(subset_sounds.index))
for j in range(0,len(subset_IT)):
  for i in range(0,len(subset_sounds)-1):
    if subset_sounds["Offset"][i] <= subset_IT["Offset"][j]:
      if pd.isna(BG_C2_JH["Breath.Group"][i]) == True:
        BG_C2_JH["Breath.Group"][i] = j+1

In [232]:
BG_C3_MD = pd.DataFrame(index=range((int(ID_freq_sound[24]))),columns=["Breath.Group"])
subset_IT = IntervalTimes[IntervalTimes["ID"] == "C3_MD"]
subset_IT.index = range(len(subset_IT.index))
subset_sounds = Offsets_all[Offsets_all["ID"] == "C3_MD"]
subset_sounds.index = range(len(subset_sounds.index))
for j in range(0,len(subset_IT)):
  for i in range(0,len(subset_sounds)-1):
    if subset_sounds["Offset"][i] <= subset_IT["Offset"][j]:
      if pd.isna(BG_C3_MD["Breath.Group"][i]) == True:
        BG_C3_MD["Breath.Group"][i] = j+1

In [233]:
BG_C4_ZO = pd.DataFrame(index=range((int(ID_freq_sound[25]))),columns=["Breath.Group"])
subset_IT = IntervalTimes[IntervalTimes["ID"] == "C4_ZO"]
subset_IT.index = range(len(subset_IT.index))
subset_sounds = Offsets_all[Offsets_all["ID"] == "C4_ZO"]
subset_sounds.index = range(len(subset_sounds.index))
for j in range(0,len(subset_IT)):
  for i in range(0,len(subset_sounds)-1):
    if subset_sounds["Offset"][i] <= subset_IT["Offset"][j]:
      if pd.isna(BG_C4_ZO["Breath.Group"][i]) == True:
        BG_C4_ZO["Breath.Group"][i] = j+1

In [234]:
BG_C5_HL = pd.DataFrame(index=range((int(ID_freq_sound[26]))),columns=["Breath.Group"])
subset_IT = IntervalTimes[IntervalTimes["ID"] == "C5_HL"]
subset_IT.index = range(len(subset_IT.index))
subset_sounds = Offsets_all[Offsets_all["ID"] == "C5_HL"]
subset_sounds.index = range(len(subset_sounds.index))
for j in range(0,len(subset_IT)):
  for i in range(0,len(subset_sounds)-1):
    if subset_sounds["Offset"][i] <= subset_IT["Offset"][j]:
      if pd.isna(BG_C5_HL["Breath.Group"][i]) == True:
        BG_C5_HL["Breath.Group"][i] = j+1

In [235]:
BG_C6_LH = pd.DataFrame(index=range((int(ID_freq_sound[27]))),columns=["Breath.Group"])
subset_IT = IntervalTimes[IntervalTimes["ID"] == "C6_LH"]
subset_IT.index = range(len(subset_IT.index))
subset_sounds = Offsets_all[Offsets_all["ID"] == "C6_LH"]
subset_sounds.index = range(len(subset_sounds.index))
for j in range(0,len(subset_IT)):
  for i in range(0,len(subset_sounds)-1):
    if subset_sounds["Offset"][i] <= subset_IT["Offset"][j]:
      if pd.isna(BG_C6_LH["Breath.Group"][i]) == True:
        BG_C6_LH["Breath.Group"][i] = j+1

In [236]:
BG_C7_SH = pd.DataFrame(index=range(int(ID_freq_sound[28])),columns=["Breath.Group"])
subset_IT = IntervalTimes[IntervalTimes["ID"] == "C7_SH"]
subset_IT.index = range(len(subset_IT.index))
subset_sounds = Offsets_all[Offsets_all["ID"] == "C7_SH"]
subset_sounds.index = range(len(subset_sounds.index))
for j in range(0,len(subset_IT)):
  for i in range(0,len(subset_sounds)-1):
    if subset_sounds["Offset"][i] <= subset_IT["Offset"][j]:
      if pd.isna(BG_C7_SH["Breath.Group"][i]) == True:
        BG_C7_SH["Breath.Group"][i] = j+1

In [237]:
BG_C9_RD = pd.DataFrame(index=range((int(ID_freq_sound[29]))),columns=["Breath.Group"])
subset_IT = IntervalTimes[IntervalTimes["ID"] == "C9_RD"]
subset_IT.index = range(len(subset_IT.index))
subset_sounds = Offsets_all[Offsets_all["ID"] == "C9_RD"]
subset_sounds.index = range(len(subset_sounds.index))
for j in range(0,len(subset_IT)):
  for i in range(0,len(subset_sounds)-1):
    if subset_sounds["Offset"][i] <= subset_IT["Offset"][j]:
      if pd.isna(BG_C9_RD["Breath.Group"][i]) == True:
        BG_C9_RD["Breath.Group"][i] = j+1

**2. Assign number of items in a BreathGroup**

In [29]:
m_24fa = pd.DataFrame()  
m = BG_24fa["Breath.Group"].value_counts().sort_index()
subset_IT = IntervalTimes[IntervalTimes["ID"] == "24fa"]
subset_IT.index = range(len(subset_IT.index))
for a in range (0,len(subset_IT)):
  m_current = pd.DataFrame(np.repeat(m.iloc[a], m.iloc[a], axis=0))
  m_24fa = m_24fa.append([m_current], ignore_index = True)
BG_24fa = pd.concat([BG_24fa, m_24fa], axis=1).astype('Int64')
BG_24fa.rename(columns = {0:'m'}, inplace = True)

In [239]:
m_24ma = pd.DataFrame()  
m = BG_24ma["Breath.Group"].value_counts().sort_index()
subset_IT = IntervalTimes[IntervalTimes["ID"] == "24ma"]
subset_IT.index = range(len(subset_IT.index))
for a in range (0,len(subset_IT)):
  m_current = pd.DataFrame(np.repeat(m.iloc[a], m.iloc[a], axis=0))
  m_24ma = m_24fa.append([m_current], ignore_index = True)
BG_24ma = pd.concat([BG_24ma, m_24ma], axis=1).astype('Int64')
BG_24ma.rename(columns = {0:'m'}, inplace = True)

In [240]:
m_24mb = pd.DataFrame()  
m = BG_24mb["Breath.Group"].value_counts().sort_index()
subset_IT = IntervalTimes[IntervalTimes["ID"] == "24mb"]
subset_IT.index = range(len(subset_IT.index))
for a in range (0,len(subset_IT)):
  m_current = pd.DataFrame(np.repeat(m.iloc[a], m.iloc[a], axis=0))
  m_24mb = m_24mb.append([m_current], ignore_index = True)
BG_24mb = pd.concat([BG_24mb, m_24mb], axis=1).astype('Int64')
BG_24mb.rename(columns = {0:'m'}, inplace = True)

In [241]:
m_26f = pd.DataFrame()  
m = BG_26f["Breath.Group"].value_counts().sort_index()
subset_IT = IntervalTimes[IntervalTimes["ID"] == "26f"]
subset_IT.index = range(len(subset_IT.index))
for a in range (0,len(subset_IT)):
  m_current = pd.DataFrame(np.repeat(m.iloc[a], m.iloc[a], axis=0))
  m_26f = m_26f.append([m_current], ignore_index = True)
BG_26f = pd.concat([BG_26f, m_26f], axis=1).astype('Int64')
BG_26f.rename(columns = {0:'m'}, inplace = True)

In [242]:
m_29ma = pd.DataFrame()  
m = BG_29ma["Breath.Group"].value_counts().sort_index()
subset_IT = IntervalTimes[IntervalTimes["ID"] == "29ma"]
subset_IT.index = range(len(subset_IT.index))
for a in range (0,len(subset_IT)):
  m_current = pd.DataFrame(np.repeat(m.iloc[a], m.iloc[a], axis=0))
  m_29ma = m_29ma.append([m_current], ignore_index = True)
BG_29ma = pd.concat([BG_29ma, m_29ma], axis=1).astype('Int64')
BG_29ma.rename(columns = {0:'m'}, inplace = True)

In [298]:
m_29mb = pd.DataFrame()  
m = BG_29mb["Breath.Group"].value_counts().sort_index()
m.index = range(len(m.index))
subset_IT = IntervalTimes[IntervalTimes["ID"] == "29mb"]
subset_IT.index = range(len(subset_IT.index))
for a in range (0,len(m)):                        
  m_current = pd.DataFrame(np.repeat(m.iloc[a], m.iloc[a], axis=0))
  m_29mb = m_29mb.append([m_current], ignore_index = True)
BG_29mb = pd.concat([BG_29mb, m_29mb], axis=1).astype('Int64')
BG_29mb.rename(columns = {0:'m'}, inplace = True)

In [251]:
m_29mc = pd.DataFrame()  
m = BG_29mc["Breath.Group"].value_counts().sort_index()
subset_IT = IntervalTimes[IntervalTimes["ID"] == "29mc"]
subset_IT.index = range(len(subset_IT.index))
for a in range (0,len(subset_IT)):
  m_current = pd.DataFrame(np.repeat(m.iloc[a], m.iloc[a], axis=0))
  m_29mc = m_29mc.append([m_current], ignore_index = True)
BG_29mc = pd.concat([BG_29mc, m_29mc], axis=1).astype('Int64')
BG_29mc.rename(columns = {0:'m'}, inplace = True)

In [252]:
m_32m = pd.DataFrame()  
m = BG_32m["Breath.Group"].value_counts().sort_index()
subset_IT = IntervalTimes[IntervalTimes["ID"] == "32m"]
subset_IT.index = range(len(subset_IT.index))
for a in range (0,len(subset_IT)):
  m_current = pd.DataFrame(np.repeat(m.iloc[a], m.iloc[a], axis=0))
  m_32m = m_32m.append([m_current], ignore_index = True)
BG_32m = pd.concat([BG_32m, m_32m], axis=1).astype('Int64')
BG_32m.rename(columns = {0:'m'}, inplace = True)

In [253]:
m_34m = pd.DataFrame()  
m = BG_34m["Breath.Group"].value_counts().sort_index()
subset_IT = IntervalTimes[IntervalTimes["ID"] == "34m"]
subset_IT.index = range(len(subset_IT.index))
for a in range (0,len(subset_IT)):
  m_current = pd.DataFrame(np.repeat(m.iloc[a], m.iloc[a], axis=0))
  m_34m = m_34m.append([m_current], ignore_index = True)
BG_34m = pd.concat([BG_34m, m_34m], axis=1).astype('Int64')
BG_34m.rename(columns = {0:'m'}, inplace = True)

In [254]:
m_35mb = pd.DataFrame()  
m = BG_35mb["Breath.Group"].value_counts().sort_index()
subset_IT = IntervalTimes[IntervalTimes["ID"] == "35mb"]
subset_IT.index = range(len(subset_IT.index))
for a in range (0,len(subset_IT)):
  m_current = pd.DataFrame(np.repeat(m.iloc[a], m.iloc[a], axis=0))
  m_35mb = m_35mb.append([m_current], ignore_index = True)
BG_35mb = pd.concat([BG_35mb, m_35mb], axis=1).astype('Int64')
BG_35mb.rename(columns = {0:'m'}, inplace = True)

In [255]:
m_46ma = pd.DataFrame()  
m = BG_46ma["Breath.Group"].value_counts().sort_index()
subset_IT = IntervalTimes[IntervalTimes["ID"] == "46ma"]
subset_IT.index = range(len(subset_IT.index))
for a in range (0,len(subset_IT)):
  m_current = pd.DataFrame(np.repeat(m.iloc[a], m.iloc[a], axis=0))
  m_46ma = m_46ma.append([m_current], ignore_index = True)
BG_46ma = pd.concat([BG_46ma, m_46ma], axis=1).astype('Int64')
BG_46ma.rename(columns = {0:'m'}, inplace = True)

In [256]:
m_50fa = pd.DataFrame()  
m = BG_50fa["Breath.Group"].value_counts().sort_index()
subset_IT = IntervalTimes[IntervalTimes["ID"] == "50fa"]
subset_IT.index = range(len(subset_IT.index))
for a in range (0,len(subset_IT)):
  m_current = pd.DataFrame(np.repeat(m.iloc[a], m.iloc[a], axis=0))
  m_50fa = m_50fa.append([m_current], ignore_index = True)
BG_50fa = pd.concat([BG_50fa, m_50fa], axis=1).astype('Int64')
BG_50fa.rename(columns = {0:'m'}, inplace = True)

In [257]:
m_50fb = pd.DataFrame()  
m = BG_50fb["Breath.Group"].value_counts().sort_index()
subset_IT = IntervalTimes[IntervalTimes["ID"] == "50fb"]
subset_IT.index = range(len(subset_IT.index))
for a in range (0,len(subset_IT)):
  m_current = pd.DataFrame(np.repeat(m.iloc[a], m.iloc[a], axis=0))
  m_50fb = m_50fb.append([m_current], ignore_index = True)
BG_50fb = pd.concat([BG_50fb, m_50fb], axis=1).astype('Int64')
BG_50fb.rename(columns = {0:'m'}, inplace = True)

In [258]:
m_54f = pd.DataFrame()  
m = BG_54f["Breath.Group"].value_counts().sort_index()
subset_IT = IntervalTimes[IntervalTimes["ID"] == "54f"]
subset_IT.index = range(len(subset_IT.index))
for a in range (0,len(subset_IT)):
  m_current = pd.DataFrame(np.repeat(m.iloc[a], m.iloc[a], axis=0))
  m_54f = m_54f.append([m_current], ignore_index = True)
BG_54f = pd.concat([BG_54f, m_54f], axis=1).astype('Int64')
BG_54f.rename(columns = {0:'m'}, inplace = True)

In [259]:
m_57m = pd.DataFrame()  
m = BG_57m["Breath.Group"].value_counts().sort_index()
subset_IT = IntervalTimes[IntervalTimes["ID"] == "57m"]
subset_IT.index = range(len(subset_IT.index))
for a in range (0,len(subset_IT)):
  m_current = pd.DataFrame(np.repeat(m.iloc[a], m.iloc[a], axis=0))
  m_57m = m_57m.append([m_current], ignore_index = True)
BG_57m = pd.concat([BG_57m, m_57m], axis=1).astype('Int64')
BG_57m.rename(columns = {0:'m'}, inplace = True)

In [260]:
m_60m = pd.DataFrame()  
m = BG_60m["Breath.Group"].value_counts().sort_index()
subset_IT = IntervalTimes[IntervalTimes["ID"] == "60m"]
subset_IT.index = range(len(subset_IT.index))
for a in range (0,len(subset_IT)):
  m_current = pd.DataFrame(np.repeat(m.iloc[a], m.iloc[a], axis=0))
  m_60m = m_60m.append([m_current], ignore_index = True)
BG_60m = pd.concat([BG_60m, m_60m], axis=1).astype('Int64')
BG_60m.rename(columns = {0:'m'}, inplace = True)

In [283]:
m_62f = pd.DataFrame()  
m = BG_62f["Breath.Group"].value_counts().sort_index()
subset_IT = IntervalTimes[IntervalTimes["ID"] == "62f"]
subset_IT.index = range(len(subset_IT.index))
for a in range (0,len(subset_IT)):
  m_current = pd.DataFrame(np.repeat(m.iloc[a], m.iloc[a], axis=0))
  m_62f = m_62f.append([m_current], ignore_index = True)
BG_62f = pd.concat([BG_62f, m_62f], axis=1).astype('Int64')
BG_62f.rename(columns = {0:'m'}, inplace = True)

In [ ]:
m_62m = pd.DataFrame()  
m = BG_62m["Breath.Group"].value_counts().sort_index()
m.index = range(len(m.index))
subset_IT = IntervalTimes[IntervalTimes["ID"] == "62m"]
subset_IT.index = range(len(subset_IT.index))
for a in range (0,len(m)):
  m_current = pd.DataFrame(np.repeat(m.iloc[a], m.iloc[a], axis=0))
  m_62m = m_62m.append([m_current], ignore_index=True)
BG_62m = pd.concat([BG_62m, m_62m], axis=1).astype('Int64')
BG_62m.rename(columns = {0:'m'}, inplace = True)

In [265]:
m_C10_KS = pd.DataFrame()  
m = BG_C10_KS["Breath.Group"].value_counts().sort_index()
subset_IT = IntervalTimes[IntervalTimes["ID"] == "C10_KS"]
subset_IT.index = range(len(subset_IT.index))
for a in range (0,len(subset_IT)):
  m_current = pd.DataFrame(np.repeat(m.iloc[a], m.iloc[a], axis=0))
  m_C10_KS = m_C10_KS.append([m_current], ignore_index = True)
BG_C10_KS = pd.concat([BG_C10_KS, m_C10_KS], axis=1).astype('Int64')
BG_C10_KS.rename(columns = {0:'m'}, inplace = True)

In [266]:
m_C11_NP = pd.DataFrame()  
m = BG_C11_NP["Breath.Group"].value_counts().sort_index()
subset_IT = IntervalTimes[IntervalTimes["ID"] == "C11_NP"]
subset_IT.index = range(len(subset_IT.index))
for a in range (0,len(subset_IT)):
  m_current = pd.DataFrame(np.repeat(m.iloc[a], m.iloc[a], axis=0))
  m_C11_NP = m_C10_KS.append([m_current], ignore_index = True)
BG_C11_NP = pd.concat([BG_C11_NP, m_C11_NP], axis=1).astype('Int64')
BG_C11_NP.rename(columns = {0:'m'}, inplace = True)

In [267]:
m_C12_CC = pd.DataFrame()  
m = BG_C12_CC["Breath.Group"].value_counts().sort_index()
subset_IT = IntervalTimes[IntervalTimes["ID"] == "C12_CC"]
subset_IT.index = range(len(subset_IT.index))
for a in range (0,len(subset_IT)):
  m_current = pd.DataFrame(np.repeat(m.iloc[a], m.iloc[a], axis=0))
  m_C12_CC = m_C12_CC.append([m_current], ignore_index = True)
BG_C12_CC = pd.concat([BG_C12_CC, m_C12_CC], axis=1).astype('Int64')
BG_C12_CC.rename(columns = {0:'m'}, inplace = True)

In [268]:
m_C13_RG = pd.DataFrame()  
m = BG_C13_RG["Breath.Group"].value_counts().sort_index()
subset_IT = IntervalTimes[IntervalTimes["ID"] == "C13_RG"]
subset_IT.index = range(len(subset_IT.index))
for a in range (0,len(subset_IT)):
  m_current = pd.DataFrame(np.repeat(m.iloc[a], m.iloc[a], axis=0))
  m_C13_RG = m_C10_KS.append([m_current], ignore_index = True)
BG_C13_RG = pd.concat([BG_C13_RG, m_C13_RG], axis=1).astype('Int64')
BG_C13_RG.rename(columns = {0:'m'}, inplace = True)

In [269]:
m_C1_DS = pd.DataFrame()  
m = BG_C1_DS["Breath.Group"].value_counts().sort_index()
subset_IT = IntervalTimes[IntervalTimes["ID"] == "C1_DS"]
subset_IT.index = range(len(subset_IT.index))
for a in range (0,len(subset_IT)):
  m_current = pd.DataFrame(np.repeat(m.iloc[a], m.iloc[a], axis=0))
  m_C1_DS = m_C1_DS.append([m_current], ignore_index = True)
BG_C1_DS = pd.concat([BG_C1_DS, m_C1_DS], axis=1).astype('Int64')
BG_C1_DS.rename(columns = {0:'m'}, inplace = True)

In [272]:
m_C2_JH = pd.DataFrame()  
m = BG_C2_JH["Breath.Group"].value_counts().sort_index()
subset_IT = IntervalTimes[IntervalTimes["ID"] == "C2_JH"]
subset_IT.index = range(len(subset_IT.index))
for a in range (0,len(subset_IT)):
  m_current = pd.DataFrame(np.repeat(m.iloc[a], m.iloc[a], axis=0))
  m_C2_JH = m_C2_JH.append([m_current], ignore_index = True)
BG_C2_JH = pd.concat([BG_C2_JH, m_C2_JH], axis=1).astype('Int64')
BG_C2_JH.rename(columns = {0:'m'}, inplace = True)

In [274]:
m_C3_MD = pd.DataFrame()  
m = BG_C3_MD["Breath.Group"].value_counts().sort_index()
subset_IT = IntervalTimes[IntervalTimes["ID"] == "C3_MD"]
subset_IT.index = range(len(subset_IT.index))
for a in range (0,len(subset_IT)):
  m_current = pd.DataFrame(np.repeat(m.iloc[a], m.iloc[a], axis=0))
  m_C3_MD = m_C3_MD.append([m_current], ignore_index = True)
BG_C3_MD = pd.concat([BG_C3_MD, m_C3_MD], axis=1).astype('Int64')
BG_C3_MD.rename(columns = {0:'m'}, inplace = True)

In [275]:
m_C4_ZO = pd.DataFrame()  
m = BG_C4_ZO["Breath.Group"].value_counts().sort_index()
subset_IT = IntervalTimes[IntervalTimes["ID"] == "C4_ZO"]
subset_IT.index = range(len(subset_IT.index))
for a in range (0,len(subset_IT)):
  m_current = pd.DataFrame(np.repeat(m.iloc[a], m.iloc[a], axis=0))
  m_C4_ZO = m_C4_ZO.append([m_current], ignore_index = True)
BG_C4_ZO = pd.concat([BG_C4_ZO, m_C4_ZO], axis=1).astype('Int64')
BG_C4_ZO.rename(columns = {0:'m'}, inplace = True)

In [276]:
m_C5_HL = pd.DataFrame()  
m = BG_C5_HL["Breath.Group"].value_counts().sort_index()
subset_IT = IntervalTimes[IntervalTimes["ID"] == "C5_HL"]
subset_IT.index = range(len(subset_IT.index))
for a in range (0,len(subset_IT)):
  m_current = pd.DataFrame(np.repeat(m.iloc[a], m.iloc[a], axis=0))
  m_C5_HL = m_C5_HL.append([m_current], ignore_index = True)
BG_C5_HL = pd.concat([BG_C5_HL, m_C5_HL], axis=1).astype('Int64')
BG_C5_HL.rename(columns = {0:'m'}, inplace = True)

In [277]:
m_C6_LH = pd.DataFrame()  
m = BG_C6_LH["Breath.Group"].value_counts().sort_index()
subset_IT = IntervalTimes[IntervalTimes["ID"] == "C6_LH"]
subset_IT.index = range(len(subset_IT.index))
for a in range (0,len(subset_IT)):
  m_current = pd.DataFrame(np.repeat(m.iloc[a], m.iloc[a], axis=0))
  m_C6_LH = m_C6_LH.append([m_current], ignore_index = True)
BG_C6_LH = pd.concat([BG_C6_LH, m_C6_LH], axis=1).astype('Int64')
BG_C6_LH.rename(columns = {0:'m'}, inplace = True)

In [278]:
m_C7_SH = pd.DataFrame()  
m = BG_C7_SH["Breath.Group"].value_counts().sort_index()
subset_IT = IntervalTimes[IntervalTimes["ID"] == "C7_SH"]
subset_IT.index = range(len(subset_IT.index))
for a in range (0,len(subset_IT)):
  m_current = pd.DataFrame(np.repeat(m.iloc[a], m.iloc[a], axis=0))
  m_C7_SH = m_C7_SH.append([m_current], ignore_index = True)
BG_C7_SH = pd.concat([BG_C7_SH, m_C7_SH], axis=1).astype('Int64')
BG_C7_SH.rename(columns = {0:'m'}, inplace = True)

In [279]:
m_C9_RD = pd.DataFrame()  
m = BG_C9_RD["Breath.Group"].value_counts().sort_index()
subset_IT = IntervalTimes[IntervalTimes["ID"] == "C9_RD"]
subset_IT.index = range(len(subset_IT.index))
for a in range (0,len(subset_IT)):
  m_current = pd.DataFrame(np.repeat(m.iloc[a], m.iloc[a], axis=0))
  m_C9_RD = m_C9_RD.append([m_current], ignore_index = True)
BG_C9_RD = pd.concat([BG_C9_RD, m_C9_RD], axis=1).astype('Int64')
BG_C9_RD.rename(columns = {0:'m'}, inplace = True)

In [ ]:
read_TextGrid_duration = pd.concat([read_TextGrid_duration, all], axis=1)

In [ ]:
read_TextGrid_duration.rename(columns = {0:'Breath.Group'}, inplace = True)

In [ ]:
read_TextGrid_duration.drop(['Unnamed: 0'], axis=1 , inplace = True)

In [ ]:
read_TextGrid_duration

,ID,Type,Sound,Onset,Offset,Duration,Breath.Group
0,24fa,silence,"""sp""",0.000,0.013,0.013,1
1,24fa,silence,"""sp""",0.013,0.143,0.130,1
2,24fa,consonant,"""S""",0.143,0.263,0.120,1
3,24fa,vowel,"""OW1""",0.263,0.343,0.080,1
4,24fa,consonant,"""HH""",0.343,0.482,0.139,1
...,...,...,...,...,...,...,...
24338,C13_RG,consonant,"""N""",87.881,87.910,0.029,33
24339,C13_RG,vowel,"""EY2""",87.910,88.081,0.171,33
24340,C13_RG,consonant,"""M""",88.081,88.191,0.110,33
24341,C13_RG,consonant,"""Z""",88.191,88.281,0.090,33


**6. Save File Created by Going Through Participants Individually**

In [ ]:
%cd /content/gdrive/MyDrive/RhythmAnalysisPipeline/

/content/gdrive/MyDrive/RhythmAnalysisPipeline


In [ ]:
dir = "2.BreathGroups_Assigned"

if os.path.exists(dir) == False:
  os.mkdir(dir)

In [ ]:
%cd /content/gdrive/MyDrive/RhythmAnalysisPipeline/2.BreathGroups_Assigned/

/content/gdrive/MyDrive/RhythmAnalysisPipeline/2.BreathGroups_Assigned


In [ ]:
read_TextGrid_duration.to_excel("reading_TextGrid_comb_BG.xlsx")

In [ ]:
%cd /content/gdrive/MyDrive/RhythmAnalysisPipeline/

/content/gdrive/MyDrive/RhythmAnalysisPipeline
